In [9]:
# from model.model_type import ModelType
from service.site_scraper import SiteScraper

gsp_scraper = SiteScraper(
    name="gsp",
    base_url="https://www.gsp.ro",
    traffic=3.6,
    time_selector="p.data-author",
    block_selector="div.news-item",
    link_selector="h2 a",
    title_strategy="text"
)

digisport_scraper = SiteScraper(
    name="digisport",
    base_url="https://www.digisport.ro",
    traffic=3.3,
    time_selector="cite",
    block_selector="article.article",
    link_selector="a.article-link, h3 a.widget-latest-list-item-link",
    title_strategy="attribute",
    title_attribute="title",
    # model=ModelType.SPACY
)

fanatik_scraper = SiteScraper(
    name="fanatik",
    base_url="https://www.fanatik.ro",
    traffic=2.9,  # Estimated, adjust as needed
    time_selector="span.date",  # Often used near article blocks
    block_selector="div.article",  # Container for each article
    link_selector="h3 a",  # Headline links
    title_strategy="text",
    # model=ModelType.GPT
)

prosport_scraper = SiteScraper(
    name="prosport",
    base_url="https://www.prosport.ro",
    traffic=3.1,
    time_selector="span.date",
    block_selector="h2.article__title",  # Directly targets the title block
    link_selector="a",  # The anchor inside the h2
    title_strategy="text",
    # model=ModelType.CLAUDE
)

sites = [gsp_scraper, digisport_scraper, fanatik_scraper, prosport_scraper]
# sites = [prosport_scraper]
total_traffic = sum(site.traffic for site in sites)

In [10]:
for site in sites:
    site.compute_weight(total_traffic)
    site.scrape_recent_articles(minutes=360)
    # site.save_to_csv()
    # site.short_print()


In [11]:
for site in sites:
    site.save_to_csv()
    # site.short_print()

In [12]:
from service.story_clusterer import StoryClusterer

clusterer = StoryClusterer(sites, 360, 0.3, 0.2)
clusterer.cluster_stories()

clusterer.print_matched_clusters()

Shared article https://www.fanatik.ro/strigat-de-disperare-pentru-ajutorarea-lui-cornel-dinu-situatia-grea-pe-care-o-traverseaza-legenda-lui-dinamo-e-cazut-psihic-21289689 on digisport - fanatik skipped https://www.fanatik.ro/strigat-de-disperare-pentru-ajutorarea-lui-cornel-dinu-situatia-grea-pe-care-o-traverseaza-legenda-lui-dinamo-e-cazut-psihic-21289689 digisport fanatik
Shared article https://www.prosport.ro/fotbal-intern/superliga/gigi-becali-l-a-facut-praf-pe-costel-galca-eu-sunt-imparatul-ii-faceam-schimbarile-din-puscarie-falsitatea-lui-m-a-deranjat-exclusiv-20292961 on digisport - prosport skipped https://www.prosport.ro/fotbal-intern/superliga/gigi-becali-l-a-facut-praf-pe-costel-galca-eu-sunt-imparatul-ii-faceam-schimbarile-din-puscarie-falsitatea-lui-m-a-deranjat-exclusiv-20292961 digisport prosport

🧠 Cluster #1 — Score: 0.519

🧠 Cluster #2 — Score: 0.504

🧠 Cluster #3 — Score: 0.496

🧠 Cluster #4 — Score: 0.481

🧠 Cluster #5 — Score: 0.256

🧠 Cluster #6 — Score: 0.256

🔍

In [14]:

top_stories = clusterer.score_clusters()

for i, story in enumerate(top_stories[:5], 1):
    print(f"\n🏆 Top {i} — Score: {story['score']}")
    for article in story["articles"]:
        print(f"- {article.site}: {article.title}")



🏆 Top 1 — Score: 0.519
- prosport: Cine transmite la TV România – Moldova
- gsp: Cine transmite la TV amicalul România - Moldova » Moment istoric la București

🏆 Top 2 — Score: 0.504
- fanatik: Ilie Dumitrescu a dat verdictul: cele 5 echipe din SuperLiga care prind 100% play-off-ul. Cine iese din schemă
- gsp: Ilie Dumitrescu a numit 5 din cele șase echipe care vor fi în play-off: „Avem așa”

🏆 Top 3 — Score: 0.496
- digisport: Adrian Petre și-a relansat cariera! 6 goluri în 7 meciuri: "Încep să-mi revin"
- prosport: Adrian Petre și-a relansat cariera! 6 goluri în 7 meciuri: "Încep să-mi revin"

🏆 Top 4 — Score: 0.481
- digisport: Lovitură pentru Barcelona? Adversara din Liga Campionilor, gata să cumpere un jucător al lui Hansi Flick
- fanatik: Veste mare pentru Dani Alves, la șapte luni după ce a fost achitat de acuzația de viol și a ieșit din închisoare

🏆 Top 5 — Score: 0.256
- digisport: Strigăt de disperare pentru ajutorarea lui Cornel Dinu. Situația grea pe care o traversează le